### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Preprocessing

In [ ]:
def read_image_file(path):
    with open(path, "rb") as f:
        data = f.read()
        num_cols = int(data[3:5])
        num_rows = int(data[6:9])
        parsed = np.frombuffer(data, dtype=np.uint8, offset=14)
        return np.array(parsed).reshape(num_rows, num_cols)

In [ ]:
# visaulize 8th individual
id = 8
plt.figure(figsize=(8,8))
for j in range(1,11):
    img = read_image_file(f"faces/s{id}/{j}.pgm")
    plt.subplot(4, 5, j)
    plt.axis('off')
    plt.imshow(img, cmap='gray')

In [ ]:
# visaulize 1st image of all individuals 
plt.figure(figsize=(40,40))
for id in range(1,41):
    img = read_image_file(f"faces/s{id}/{1}.pgm")
    plt.subplot(10,10,id)
    plt.axis('off')
    plt.imshow(img, cmap='gray')
plt.tight_layout()
plt.show()

In [ ]:
# 1-40 individuals, 10 images each, each image is 112x92

# Data Matrix
D = np.empty((0, 112*92), dtype=float) # number of images [rows] x number of pixels [columns]
# label vector
y = np.array([], dtype=int)

for id in range(1, 41):
    for j in range(1, 11):
        img = read_image_file(f"faces/s{id}/{j}.pgm")
        D = np.vstack((D, img.reshape(1, -1).astype(float)))
        y = np.append(y,id)

# shape of Data Matrix and label vector
print(D.shape, y.shape)

In [ ]:
# splitting dataset into training and testing sets
odd_rows = np.array([i % 2 != 0 for i in range(400)]) 

# odds rows for taining set
D_train = D[odd_rows]
y_train = y[odd_rows]

# even rows for testing set
D_test = D[~ odd_rows]
y_test = y[~ odd_rows]

# shapes of resulting dataset
print(D_train.shape, y_train.shape)
print(D_test.shape, y_test.shape)

### PCA

In [ ]:
class PCA():
    def __init__(self, Data, eigen_values=None, eigen_vectors=None):
        """
        Initialize PCA with data and optional eigenvalues and eigenvectors.
        :param Data: The data matrix.
        :param eigen_values: Optional eigenvalues.
        :param eigen_vectors: Optional eigenvectors.
        """
        self.CenteredData = Data - np.mean(Data, axis=0)
        
        if eigen_values is not None and eigen_vectors is not None:
            self.eigen_values = eigen_values
            self.eigen_vectors = eigen_vectors
        else:
            self.calc_eig()

    def project(self, alpha=0.85):
        """
        Project the data into the PCA space.
        :param alpha: The percentage of variance to be explained.
        :return: The projected data.
        """
        # check if alpha is between 0 and 1
        if alpha < 0 or alpha > 1:
            raise ValueError("alpha must be between 0 and 1")

        # count PCs from aplha
        num_components = self.count_num_components(alpha)

        # get projection matrix with num_components
        projection_mat = self.eigen_vectors[:,:num_components]

        # compute projected data
        return self.CenteredData @ projection_mat
    
    def calc_eig(self):
        """
        Calculate the eigenvalues and eigenvectors of the covariance matrix.
        :return: The sorted eigenvalues and eigenvectors.
        """
        # covariance matrix
        cov_mat = np.cov(self.CenteredData, rowvar=False)

        # eigenvalues and eigenvectors
        self.eigen_values, self.eigen_vectors = np.linalg.eigh(cov_mat) 

        # sort indices
        sort_idx = np.argsort(self.eigen_values)[::-1]
        self.eigen_values  = self.eigen_values[sort_idx]
        self.eigen_vectors = self.eigen_vectors[:, sort_idx]

    def count_num_components(self, alpha):
        """
        Count the number of components needed to explain a certain percentage of variance.
        :param alpha: The percentage of variance to be explained.
        :return: The number of components needed.
        """
        total_var = np.sum(self.eigen_values)
        cumvar = np.cumsum(self.eigen_values) / total_var

        num_components = np.searchsorted(cumvar, alpha) + 1
        return num_components

In [ ]:
# initialize PCA with no loaded eigenvalues and eigenvectors
pca_first = PCA(D_train)
# eigens
eigen_values, eigen_vectors = pca_first.eigen_values, pca_first.eigen_vectors

# save eigen values and vectors
np.save("eigen_values.npy", eigen_values)
np.save("eigen_vectors.npy", eigen_vectors)

In [ ]:
# load eigen values and vectors
eigen_values = np.load("eigen_values.npy")
eigen_vectors = np.load("eigen_vectors.npy")

# initialize PCA with loaded eigen values and vectors
pca = PCA(D_train, eigen_values, eigen_vectors)

# check dimensions of eigen values and vectors
print(eigen_values.shape, eigen_vectors.shape)

In [ ]:
# PCA with different aplhas
alphas = [.8, .85, .9, .95]
# D_projections {alpha: projected_data} 
D_projections = {}

for alpha in alphas:
    D_projections[alpha] = pca.project(alpha)
    print(f"Dimensions at alpha= {alpha} : {D_projections[alpha].shape}")

In [ ]:
# reconstruct images of some samples with different number of components
sample_faces = [0, 5]
num_components = [10, 50, 100, 150, 200]
reconstruct_images = {}
j = 0
for  k in sample_faces:
    for i in num_components:
        img = D_train[k] @ (eigen_vectors[:,:i] @ eigen_vectors[:,:i].T) + np.mean(D_train, axis=0)
        reconstruct_images[(k,i)] = img
        j = j + 1

In [ ]:
# visaulize reconstucted faces in PCA space
plt.figure(figsize=(10,10))
j = 1
for key, img in reconstruct_images.items():
        plt.subplot(5,5,j)
        plt.axis('off')
        plt.title(f"sample face {key[0]+1}\n{key[1]} components",fontsize=10)
        plt.imshow(img.reshape(112,92), cmap='gray')
        j = j + 1
plt.tight_layout()

In [ ]:
# Display the Eigenfaces with tup 10 eigen values
plt.figure(figsize=(20, 20))
for i in range(10):
    plt.subplot(4, 5, i + 1)
    plt.axis('off')
    plt.imshow(eigen_vectors[:, i].reshape(112, 92), cmap='gray')
    plt.title(f"Eigenface {i + 1} ({eigen_values[i]:.2f})")
plt.tight_layout()
plt.show()

### KMeans

In [ ]:
np.random.seed(42)

class KMeans():
    def __init__(self, K, rand_start=20):
        self.K = K
        self.rand_start = rand_start
    
    def setK(self, K):
        self.K = K

    def fit(self, D, y):
        r_best = np.array([])
        min_inertia = np.inf

        # apply kmeans (rand_start) times
        for i in range(self.rand_start):
            # set K random centeriods: centers (K, # features)
            centers = np.random.rand(self.K, D.shape[1]) * np.random.randint(np.max(D))

            # intialize responsibility matrix(1-hot encode) with zeroes: r (# samples, k)
            r = np.zeros((D.shape[0], self.K), dtype=bool)
            
            r, centers = self.kmeans_loop(D, r, centers)

            # evaluate best cluster using interia
            err = self.inertia(D, r, centers)
            if  err < min_inertia:
                r_best = r
                min_inertia = err
        
        return r_best
        
    def kmeans_loop(self, D, r, centers):
        r_prev = r.copy()

        while True:
            # compute Distance matrix for each data point x=D[i] with each center
            # assign cluster for each data point
            for i in range(len(D)):
                cluster = np.argmin(np.linalg.norm(centers - D[i], axis=1))
                r[i] = [j == cluster for j in range(self.K)]

            # test for convergence
            if np.array_equal(r_prev, r):
                break
            else:
                r_prev = r.copy()
            
            # refitting center of each cluster
            for j in range(self.K):
                centers[j] = np.mean(D[r[:,j]], axis=0)
        
        return r, centers
    
    def inertia(self, D, r, centers):
        # sum of Euclidean dist from each point to its cluster
        sum = 0
        for j in range(self.K):
            cluster_dis = np.linalg.norm(D[r[:,j]] - centers[j], axis=1)
            sum = sum + np.sum(cluster_dis)

        return sum
            

### GMM

In [ ]:
# Get max and min values in each D_projection
range_values = {}
for alpha in alphas:
    range_values[alpha] = (np.min(D_projections[alpha]), np.max(D_projections[alpha]))
    print(f"Range of values at alpha= {alpha} : {range_values[alpha]}")

In [ ]:
# Standardize the projections
D_projections_standardized = {}
for alpha in alphas:
    D_projections_standardized[alpha] = (D_projections[alpha] - np.mean(D_projections[alpha], axis=0)) / np.std(D_projections[alpha], axis=0)

In [ ]:
Σs = np.array([np.eye(200)] * 4)
print(Σs.shape)

In [ ]:
class GMM():
    def __init__(self, clusters = 40, max_iter = 100, random_starts = 10, tol = 1e-4):
        """
        Initialize GMM with number of clusters, max iterations and random starts.
        :param clusters: The number of clusters.
        :param max_iter: The maximum number of iterations.
        :param random_starts: The number of random starts.
        :param tol: The tolerance for convergence.
        """
        self.tol = tol
        self.max_iter = max_iter
        self.clusters = clusters
        self.random_starts = random_starts

    def fit(self, X):
        """
        Fit the GMM to the data.
        :param X: The data matrix.
        """
        self.n_samples, self.n_features = X.shape

        self.πs = np.ones(self.clusters) / self.clusters
        self.Σs = np.array([np.eye(self.n_features)] * self.clusters)
        
        # best random start
        best_ll = -np.inf
        best_μs = None
        best_Σs = None
        best_πs = None

        for _ in range(self.random_starts):
            
            # initialize means randomly
            self.μs = np.random.randn(self.clusters, self.n_features)

            ll = 0 # log likelihood
            
            for _ in range(self.max_iter):
                # E-step
                rs = self.e_step(X)

                # M-step
                self.m_step(X, rs)

                # calculate the new log likelihood
                nll = np.sum(np.log(np.sum(rs, axis=1) + 1e-10)) # to avoid log(0)
                if abs(nll - ll) < self.tol:
                    break

                ll = nll

            if ll > best_ll:
                best_ll = ll
                best_μs = self.μs.copy()
                best_Σs = self.Σs.copy()
                best_πs = self.πs.copy()


        # set the best parameters
        self.μs = best_μs
        self.Σs = best_Σs
        self.πs = best_πs

        print("Model fitted successfully.")

    def e_step(self, X):
        """
        E-step of the EM algorithm.
        :param X: The data matrix.
        :return: The responsibilities.
        """
        rs = np.zeros((self.n_samples, self.clusters))

        # calculate responsibilities
        for k in range(self.clusters):
            rs[:, k] = self.πs[k] * self.multivariate_gaussian(X, self.μs[k], self.Σs[k])

        # normalize responsibilities
        rs /= rs.sum(axis=1, keepdims=True) + 1e-10 # to avoid division by zero

        return rs

    def m_step(self, X, rs):
        """
        M-step of the EM algorithm.
        :param X: The data matrix.
        :param rs: The responsibilities.
        """

        #print shapes

        # calculate sum of responsibilities per cluster
        Nk = rs.sum(axis=0)
        # update weights of clusters
        self.πs = Nk / self.n_samples
        # update means of clusters
        self.μs = rs.T @ X / Nk[:, np.newaxis]

        # print(f"X shape: {X.shape}")
        # print(f"rs shape: {rs.shape}")
        # print(f"πs shape: {self.πs.shape}")
        # print(f"μs shape: {self.μs.shape}")
        # print(f"Σs shape: {self.Σs.shape}")
        # print(f"Nk shape: {Nk.shape}")

        # update covariance matrices of clusters
        self.Σs = np.zeros((self.clusters, self.n_features, self.n_features))
        for k in range(self.clusters):
            diff = X - self.μs[k]
            self.Σs[k] = (rs[:, k][:, np.newaxis] * diff).T @ diff / Nk[k]

    def multivariate_gaussian(self, X, μ, Σ):
        """
        Multivariate Gaussian distribution.
        :param X: The data matrix.
        :param μ: The mean vector.
        :param Σ: The covariance matrix.
        :return: The pdf.
        """
        d = X.shape[1]
        det_Σ = np.linalg.det(Σ)
        
        first_term = 1 / ((2 * np.pi) ** (d / 2) * np.sqrt(det_Σ))
        
        pdfs = np.zeros(X.shape[0], dtype=float)

        Σ_inv = np.linalg.inv(Σ)
        for i in range(X.shape[0]):
            diff = (X[i] - μ)[:, np.newaxis]
            second_term = np.exp(-0.5 * diff.T @ Σ_inv @ diff)
            print(diff.T @ Σ_inv @ diff)
            pdfs[i] = first_term * second_term

        return pdfs


In [ ]:
gmm = GMM(clusters=40, max_iter=100, random_starts=10, tol=1e-4)
gmm.fit(D_projections[0.85])